In [ ]:
# Import Libraries
import numpy as np
import torch 
from genie.action import LatentAction
from accelerate import Accelerator
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# Training Hyperparameters
lr = 0.001
batch_size = 8
n_epochs = 10

In [ ]:
# Load Data
data = ...

In [ ]:
# Model Architecture
ENC_BLUEPRINT = (
    ('space-time_attn', {
        'n_rep' : 2,
        'n_embd' : 256,
        'n_head' : 4,
        'd_head' : 8,
    }),
    ('spacetime_downsample', {
        'in_channels' : 256,
        'kernel_size' : 3,
        'time_factor' : 1,
        'space_factor' : 2,
    }),
    ('space-time_attn', {
        'n_rep' : 2,
        'n_embd' : 256,
        'n_head' : 4,
        'd_head' : 8,
    }),
)

DEC_BLUEPRINT = (
    ('space-time_attn', {
        'n_rep' : 2,
        'n_embd' : 256,
        'n_head' : 4,
        'd_head' : 16,
        'has_ext' : True,
        'time_attn_kw'  : {'key_dim' : 8},
    }),
    ('spacetime_upsample', {
        'in_channels' : 256,
        'kernel_size' : 3,
        'time_factor' : 1,
        'space_factor' : 2,
    }),
    ('space-time_attn', {
        'n_rep' : 2,
        'n_embd' : 256,
        'n_head' : 4,
        'd_head' : 16,
        'has_ext' : True,
        'time_attn_kw'  : {'key_dim' : 8},
    }),
)


In [ ]:
# Create Model

accelerator = Accelerator()

model = LatentAction(
    enc_desc = ENC_BLUEPRINT,
    dec_desc = DEC_BLUEPRINT,
    d_codebook=8,      
    inp_channels=1,     
    inp_shape=(16, 16), 
    n_embd=256, 
)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model, optimizer, data = accelerator.prepare(model, optimizer, data)

model.train()

In [ ]:
def train(model, data, optimizer, accelerator, n_epochs):
    for epoch in range(n_epochs):
        for batch in data:
            loss = model(batch)

            for param in model.parameters():
                param.grad = None

            accelerator.backward(loss)
            optimizer.step()